In [18]:
import s3fs
import sys
sys.path.append('../util')
import util_data_access as da
from tqdm.auto import tqdm
import numpy as np
import os, glob
import pandas as pd 

import sys
sys.path.append('../')
import spark_processing_scripts.util_spark as sus
import spark_processing_scripts.util_general as sug

fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'http://s3.dev.obdc.bcs.bloomberg.com'})

In [2]:
fs.ls('aspangher/edit-pathways/')

['aspangher/edit-pathways/csvs',
 'aspangher/edit-pathways/db-dumps',
 'aspangher/edit-pathways/dbs',
 'aspangher/edit-pathways/output_for_sheena',
 'aspangher/edit-pathways/pkls',
 'aspangher/edit-pathways/pqs',
 'aspangher/edit-pathways/sample_files_for_sheena',
 'aspangher/edit-pathways/spacy',
 'aspangher/edit-pathways/spark_output_final',
 'aspangher/edit-pathways/spark_processing_scripts-output',
 'aspangher/edit-pathways/spark_processing_scripts-output_sentences',
 'aspangher/edit-pathways/temp',
 'aspangher/edit-pathways/tmp']

# Download Files

In [3]:
db_name = 'guardian'
if not os.path.exists('%s_output' % db_name):
     os.makedirs('%s_output' % db_name)

In [4]:
for f in tqdm(fs.ls('aspangher/edit-pathways/spark_processing_scripts-output/%s/' % db_name)):
    f = '/'.join(f.split('/')[1:])
    fname = f.split('/')[-1]
    
    da.download_file(db_name + '_output/' + fname, f)

  0%|          | 0/464 [00:00<?, ?it/s]

In [5]:
# da.download_file('guardian.db', 'edit-pathways/dbs/newssniffer-guardian.db.gz')
# ! gunzip guardian.db.gz
# fs.ls('aspangher/edit-pathways/spark_processing_scripts-output_sentences/nyt')

# Read Files

In [6]:
dfs = []
for f in tqdm(glob.glob('%s_output/*' % db_name)):
    df = pd.read_csv(f, compression=None, index_col=0)
    dfs.append(df)

  0%|          | 0/464 [00:00<?, ?it/s]

In [7]:
full_diffs_df = pd.concat(dfs)

In [8]:
full_diffs_df[['entry_id', 'version_x', 'version_y']].drop_duplicates().shape

(608386, 3)

In [9]:
full_diffs_df.head()

,entry_id,version_x,version_y,sent_idx_x,sent_idx_y,avg_sentence_distance_x,avg_sentence_distance_y
0,1413809,2,3,9.0,NaN,NaN,NaN
1,1412764,2,3,4.0,4.0,4.500885e-07,4.500885e-07
2,1410427,6,7,10.0,10.0,5.393629e-07,5.393629e-07
3,1413523,1,2,20.0,20.0,0.000000e+00,0.000000e+00
4,1412603,3,4,10.0,10.0,3.081464e-07,3.081464e-07


In [84]:
(full_diffs_df
 .assign(c=1)
 .groupby(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])['c']
 .sum()
 .value_counts()
)

1    11395253
Name: c, dtype: int64

# Check Against Expected

In [23]:
remote_name = sug.conn_mapper_dict[db_name]
da.download_file('%s.db.gz' % db_name, 'edit-pathways/dbs/%s.db.gz' % remote_name)
! gunzip $db_name\.db\.gz

In [35]:
import sqlite3

con = sqlite3.connect('%s.db' % db_name)
# full_df = pd.read_sql('select * from entryversion where num_versions > 1 and num_versions < 40 ', con=con)
eligible_ids = pd.read_sql('select entry_id, version from entryversion where num_versions > 1 and num_versions < 40 ', con=con)

In [36]:
expected = []
for entry_id, versions in (
    eligible_ids
        .groupby('entry_id')
        .aggregate(list)['version']
        .iteritems()
):
    for version_pair in zip(versions[:-1], versions[1:]):
        expected.append({'entry_id': entry_id, 'version_pair': version_pair})

expected_df = pd.DataFrame(expected)

In [37]:
expected_df.shape

(612059, 2)

In [38]:
(full_diffs_df
 [['entry_id', 'version_x', 'version_y']]
 .drop_duplicates()
 .shape
)

(608386, 3)

In [39]:
(full_diffs_df
 [['entry_id']]
 .drop_duplicates()
 .shape
)

(230321, 1)

In [40]:
expected_df['entry_id'].drop_duplicates().shape

(231051,)

In [71]:
all_dfs = []
for f in tqdm(glob.glob('%s_pqs/*' % db_name)):
    df = pd.read_parquet(f)
    all_dfs.append(df)

  0%|          | 0/12 [00:00<?, ?it/s]

In [73]:
all_df = pd.concat(all_dfs)

In [ ]:
output_full_diffs_df = full_diffs_df

In [5]:
output_full_diffs_df = (full_diffs_df
 .fillna('nan')
 .drop_duplicates(['entry_id', 'version_x', 'version_y', 'sent_idx_x', 'sent_idx_y'])
 .replace(to_replace='nan', value=np.nan)
)

In [9]:
output_full_diffs_df.head()

,entry_id,version_x,version_y,sent_idx_x,sent_idx_y,avg_sentence_distance_x,avg_sentence_distance_y
0,25115,2,3,10.0,10.0,0.000000,0.000000
1,24640,5,6,6.0,6.0,0.000000,0.000000
2,24774,6,7,1.0,1.0,0.000000,0.000000
3,25952,13,14,27.0,27.0,0.000000,0.000000
4,24870,2,3,12.0,11.0,0.269482,0.269482


In [6]:
## final files
import sqlite3
with sqlite3.connect('%s-matched-sentences.db' % db_name) as con:
    output_full_diffs_df.to_sql('matched_sentences', con=con, index=False, chunksize=10000, if_exists='replace')

In [ ]:
da.upload_file('2021-05-19__partial-nyt-output.pkl', 'edit-pathways/output_for_sheena/2021-05-25__partial-nyt-matched-output.pkl')

In [ ]:
## intermediate files
fname = '2021-05-26__newssniffer-bbc-diffs-output.pkl'
output_full_diffs_df.to_pickle(fname, compression='gzip', chunksize=10000)

In [34]:
da.upload_file(fname, 'edit-pathways/output_for_sheena/%s' % fname)

True

In [36]:
da.upload_file('nyt_sent_output/df_nyt__start_0__end_20000__num_1.pkl', 'edit-pathways/output_for_sheena/df_nyt__start_0__end_20000__num_1.pkl')

True

# Sentences

In [22]:
## sentences

In [7]:
if not os.path.exists('%s_sent_output' % db_name):
    os.makedirs('%s_sent_output' % db_name)

In [9]:
for f in fs.ls('aspangher/edit-pathways/spark_processing_scripts-output_sentences/%s' % db_name):
    f = '/'.join(f.split('/')[1:])
    fname = f.split('/')[-1]
    da.download_file('%s_sent_output/%s' % (db_name, fname) , f)

In [10]:
con = sqlite3.connect('%s-matched-sentences.db' % db_name)
con.execute('DROP TABLE IF EXISTS split_sentences;')

In [12]:
import pandas as pd 
import pickle
import glob
from tqdm.auto import tqdm

sent_dfs = []
for f in tqdm(glob.glob('%s_sent_output/*' % db_name)):
    sent_df = pd.read_pickle(f, compression='gzip')
    ## final files
    import sqlite3
    with sqlite3.connect('%s-matched-sentences.db' % db_name) as con:
        sent_df.to_sql('split_sentences', con=con, index=False, chunksize=5000, if_exists='append')
#     break
#     sent_dfs.append(sent_df)

  0%|          | 0/9 [00:00<?, ?it/s]

In [13]:
con = sqlite3.connect('%s-matched-sentences.db' % db_name)

In [15]:
pd.read_sql('SELECT count(distinct entry_id) from split_sentences', con=con)

,count(distinct entry_id)
0,179961


In [16]:
pd.read_sql('SELECT count(distinct entry_id) from matched_sentences', con=con)

,count(distinct entry_id)
0,179938


In [17]:
! gzip bbc-2-matched-sentences.db

In [18]:
dir_name = 'spark_output_final'
dir_name = 'output_for_sheena'
da.upload_file('%s-matched-sentences.db.gz' % db_name, 'edit-pathways/%s/%s-matched-sentences.db.gz' % (dir_name, db_name))

True

In [ ]:
# ! aws s3api put-object-acl --bucket aspangher --key edit-pathways/spark_output_final/nyt-matched-sentences.db.gz --acl public-read --endpoint http://s3.dev.obdc.bcs.bloomberg.com 

In [26]:
calc_entry_versions = (full_diffs_df
 .set_index('entry_id')[['version_x', 'version_y']]
 .unstack()
 .to_frame('version')
 .reset_index()
 .drop('level_0', axis=1)
 .drop_duplicates()
)

In [29]:
sent_df[['entry_id', 'version']].drop_duplicates().shape

(79292, 2)

In [27]:
(sent_df[['entry_id','version']]
 .drop_duplicates()
 .merge(
     calc_entry_versions, 
     right_on=['entry_id', 'version'], 
     left_on=['entry_id', 'version'], 
     how='inner'
 ).shape
)

(71073, 2)

# Look at fetching operation

In [16]:
sys.path.append('../spark_processing_scripts')
import util_general as ug

In [21]:
db_name = 'bbc-2'
t = ug._download_prefetched_data_csv(db_name, False, True)

  0%|          | 0/360 [00:00<?, ?it/s]

In [24]:
to_fetch_df = ug.download_pq_to_df(db_name, t)

In [30]:
db_name

'bbc-2'

In [29]:
to_fetch_df

[]

In [62]:
import re
db_name = 'guardian'
prefetched_entry_id_list = t.values
fname = ug.conn_mapper_dict[db_name]
file_list = ug.get_fs().ls(ug.s3_pq_dir)
file_pattern = re.compile(r'%s-\d+.pq' % fname)
file_list = list(filter(lambda x: re.search(file_pattern, x), file_list))

if False:
    all_full_dfs = []
    for f_idx, fname in enumerate(file_list):
        with ug.get_fs().open(fname) as f:
            full_df = pd.read_parquet(f)

        print(f_idx)
        print('pre-filtering')
        print(full_df.shape)
        print(full_df['entry_id'].drop_duplicates().shape)
        all_full_dfs.append(full_df.copy())
        full_df = full_df.loc[lambda df: ~df['entry_id'].isin(prefetched_entry_id_list)]
        print('post-filtering')
        print(full_df.shape)
        print(full_df['entry_id'].drop_duplicates().shape)
        if len(full_df['entry_id'].drop_duplicates()) > 50:
            print('here')#full_df

In [46]:
full_df = pd.concat(all_full_dfs)

In [48]:
full_df.shape

(714873, 12)

In [49]:
full_df['entry_id'].drop_duplicates().shape

(180000,)

In [25]:
df = ug.get_rows_to_process_df(
    500, 0, t, to_fetch_df
)

AttributeError: 'list' object has no attribute 'loc'

In [ ]:
full_db = sug.download_pq_to_df('nyt')

In [47]:
prefetched_df = sug.download_prefetched_data('nyt', split_sentences=False)

In [74]:
t = (prefetched_df
 .set_index('entry_id')[['version_x', 'version_y']]
 .unstack()
 .to_frame('version')
 .reset_index()
 .drop('level_0', axis=1)
 .drop_duplicates()
)

In [88]:
t2 = full_db[['entry_id', 'version']].drop_duplicates()

In [96]:
prefetched_df['entry_id'].shape

(2560285,)

In [100]:
'entry_id' in prefetched_df

True

In [59]:
full_db['entry_id'].drop_duplicates().shape

(20000,)

In [ ]:
df = sug.get_rows_to_process_df(
    500, 0, prefetched_df, full_db
)

In [1]:
import sqlite3
import pandas as pd 
with sqlite3.connect('nyt.db') as con:
    full_nyt_df = pd.read_sql('''
        SELECT * FROM entryversion
        WHERE num_versions > 1
        AND num_versions < 40
    ''', con=con)

In [ ]:
full_nyt_df.loc[lambda df: ~df['entry_id'].isin(full_db['entry_id'])].shape

In [107]:
full_nyt_df.loc[lambda df: df['entry_id'].isin(full_db['entry_id'])].shape

(79294, 12)

# Prepare data files

In [59]:
db_name = 'guardian'
remote_name = sug.conn_mapper_dict[db_name]

# da.download_file('newssniffer-washpo.db.gz', 'edit-pathways/dbs/newssniffer-washpo.db.gz')
if not (os.path.exists('%s.db.gz' % remote_name) or os.path.exists('%s.db' % remote_name)):
    da.download_file('%s.db.gz' % db_name, 'edit-pathways/dbs/%s.db.gz' % remote_name)
    ! gunzip $db_name\.db\.gz

In [60]:
import sqlite3
with sqlite3.connect('%s.db' % db_name) as con:
    entry_ids = pd.read_sql('select DISTINCT entry_id from entryversion', con=con)['entry_id']
#     full_df = pd.read_sql('select * from entryversion', chunksize=5000, con=con)

In [53]:
if not os.path.exists('%s_pqs' % db_name):
    os.makedirs('%s_pqs' % db_name)

In [55]:
chunk_size = 20000
for chunk_idx, (s_idx, e_idx) in tqdm(enumerate(
    zip(
        range(0, len(entry_ids), chunk_size), 
        range(chunk_size, len(entry_ids) + chunk_size, chunk_size)
    )
)):
    chunk_ids = entry_ids[s_idx: e_idx].values.tolist()
    with sqlite3.connect('%s.db' % db_name) as con:
        chunk_df = pd.read_sql('''
                                SELECT * FROM entryversion
                                WHERE entry_id IN (%s)
        ''' % ', '.join(list(map(str, chunk_ids))), con=con)

    (chunk_df
     .to_parquet('%(db_name)s_pqs/%(db_name)s-%(num)s.pq' % ({'db_name': db_name, 'num': chunk_idx + 1}))
    )

|          | 0/? [00:00<?, ?it/s]

In [37]:
import os 
for f in tqdm(glob.glob('%s_pqs/*' % db_name)):
    fname = os.path.basename(f)
    da.upload_file(f, 'edit-pathways/pqs/' + fname)

  0%|          | 0/2 [00:00<?, ?it/s]

# Progress
                                edits       sentences
ap.db                           in-prog
bbc.db                          ~~~~~ 
calgaryherald.db              
canadaland.db
cbc.db
cnn.db
dailymail.db
fox.db
globemail.db
lapresse.db
nationalpost.db
newssniffer-bbc.db.gz          in-prog             
newssniffer-guardian.db.gz     in-prog
newssniffer-independent.db
newssniffer-nytimes.db.gz      x           x 
newssniffer-washpo.db          in-prog
reuters.db.gz                  x 
telegraph.db 
therebel.db
torontostar.db
torontosun.db